In [1]:
import torch.nn as nn
import torch.optim as optim
from classes import SpeedEstimatorRNN, VehicleSpeedDataset
from torch.utils.data import DataLoader

In [2]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! You can use a GPU for training.")
    print("Number of GPUs available:", torch.cuda.device_count())
    print("Current GPU being used:", torch.cuda.current_device())
    print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. Training will be performed on the CPU.")

CUDA is available! You can use a GPU for training.
Number of GPUs available: 1
Current GPU being used: 0
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
# Set dataset path
data_path = r'C:\my files\thesis\AI_training\data\training_data\training_data_noise_extended'
extension = "*.csv"

# Hyperparameters that will alter throughout the model creations
input_size = 26  # Number of CAN signals per timestep
hidden_size = [256]
num_layers = [4]
learning_rate = [0.0001]
# num of sequences in one batch
batch_size = [128]
dropout_rate = [0.2]
sequence_length = [800]


# parameters of the simulation
step_size = 10 # what the overlap between the sequences should look like in the extracted dataset
output_size = 1
num_epochs = 20

num_models = 1

default_location = r"C:\my files\thesis\AI_training\Simple RNN\trained_models\iteration_6\model_"

In [6]:
range(num_models)

range(0, 1)

In [4]:
# Training loops
for j in range(num_models):
    # Load dataset and DataLoader
    dataset = VehicleSpeedDataset(data_path, extension, seq_length = sequence_length[j], step_size = step_size)
    dataloader = DataLoader(dataset, batch_size=batch_size[j], shuffle=True, num_workers= 6, pin_memory=True)

    # Initialize model, loss function, and optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SpeedEstimatorRNN(input_size, hidden_size[j], num_layers[j], output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate[j])

    for epoch in range(num_epochs):
        total_loss = 0

        for batch_idx, (features, speeds) in enumerate(dataloader):
            features, speeds = features.to(device), speeds.to(device)

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, speeds)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Model: {j}, Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

    torch.save({
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "sequence_length": sequence_length,
        "input_size": input_size,
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "output_size": output_size,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs
    }, default_location + str(j) + ".pt")

Model: 0, Epoch [1/20], Loss: 132.6036
Model: 0, Epoch [2/20], Loss: 56.2399
Model: 0, Epoch [3/20], Loss: 25.6993
Model: 0, Epoch [4/20], Loss: 14.0145
Model: 0, Epoch [5/20], Loss: 7.3529
Model: 0, Epoch [6/20], Loss: 3.9149
Model: 0, Epoch [7/20], Loss: 2.1530
Model: 0, Epoch [8/20], Loss: 1.2369
Model: 0, Epoch [9/20], Loss: 0.7287
Model: 0, Epoch [10/20], Loss: 0.4376
Model: 0, Epoch [11/20], Loss: 0.2651
Model: 0, Epoch [12/20], Loss: 0.1912
Model: 0, Epoch [13/20], Loss: 0.1186
Model: 0, Epoch [14/20], Loss: 0.1071
Model: 0, Epoch [15/20], Loss: 0.0695
Model: 0, Epoch [16/20], Loss: 0.0708
Model: 0, Epoch [17/20], Loss: 0.0591
Model: 0, Epoch [18/20], Loss: 0.0553
Model: 0, Epoch [19/20], Loss: 0.0502
Model: 0, Epoch [20/20], Loss: 0.0462


In [1]:
#To change the format of the created models
import torch
from classes import SpeedEstimatorRNNModified  # Import your model structure

# Define paths
num_models = 12  # Total number of models
input_size = 14  # Define these based on your training setup
output_size = 1
hidden_size = [64, 64, 64, 128, 128, 256, 128, 128, 128, 128, 128, 128]  # Same as during training
num_layers = [1, 2, 3, 1, 3, 3, 2, 2, 2, 2, 4, 3]  # Model config during training

# Path to existing models and new .pt files
old_model_location = r"C:\work\AI_training\Simple RNN\trained_models\iteration_3\model_"
new_save_location = r"C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_"  # New path to save the .pt files

# Convert each model
for i in range(num_models):
    # Define model architecture
    model = SpeedEstimatorRNNModified(input_size, hidden_size[i], num_layers[i], output_size)

    # Load trained weights
    checkpoint = torch.load(old_model_location + str(i), map_location=torch.device('cpu'))  # Adjust path as needed
    model.load_state_dict(checkpoint["model_state_dict"])  # Load the weights
    model.eval()  # Set model to evaluation mode

    # Save the model in .pt format
    torch.save(model, new_save_location + str(i) + ".pt")
    print(f"Model {i} saved in .pt format at {new_save_location + str(i) + '.pt'}")

Model 0 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_0.pt
Model 1 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_1.pt
Model 2 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_2.pt
Model 3 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_3.pt
Model 4 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_4.pt
Model 5 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_5.pt
Model 6 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_6.pt
Model 7 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_7.pt
Model 8 saved in .pt format at C:\work\AI_training\Simple RNN\tr